# Praktikum IF3270 2023/2024

Tujuan praktikum IF3270 Pembelajaran Mesin:
1.   Peserta memahami rangkaian proses analitik data menggunakan pendekatan pembelajaran mesin.
2.   Peserta memahami bahwa proses pengembangan model pembelajaran mesin juga ditentukan dari kualitas data, penanganan data, dan penentuan algoritma serta hyperparameter-nya; tidak cukup hanya dengan memastikan implementasi algoritma berjalan tanpa kesalahan.
3.   Peserta mampu menginterpretasikan hasil dari evaluasi model dalam proses analitik menggunakan pendekatan pembelajaran mesin.

Praktikum dilaksanakan secara berkelompok. Setiap kelompok terdiri atas 2 mahasiswa. Perhatikan bahwa terdapat berkas yang harus dikumpulkan saat keberjalanan praktikum untuk bagian A (25 April 2024, pukul 12.00 WIB) dan berkas yang dikumpulkan setelah waktu praktikum selesai untuk bagian B (25 April 2024, pukul 21.00 WIB).

Disediakan data yang sudah dibagi menjadi data latih (`df_train`), data validasi (`df_val`), dan data uji (`df_test`).

**Bagian 1**: (batas waktu: 25 April 2024, 12.00 WIB)

1. Buatlah _baseline_ dengan menggunakan model _logistic regression_.
2. Lakukan analisis data terkait hal berikut:
    - _duplicate value_,
    - _missing value_,
    - _outlier_,
    - _balance of data_.
3. Jelaskan rencana penanganan yang ada pada poin 2.
4. Jelaskan teknik _encoding_ yang digunakan terhadap data yang disediakan apabila dilakukan, disertai dengan alasan.
5. Buatlah desain eksperimen dengan menentukan hal berikut:
    - tujuan eksperimen,
    - variabel dependen dan independen,
    - strategi eksperimen,
    - skema validasi.
    
**Bagian 2**: (batas waktu: 25 April 2024, 21.00 WIB)

6. Implementasikan strategi eksperimen dan skema validasi yang telah ditentukan pada poin 5.
7. Berdasarkan hasil prediksi yang dihasilkan, buatlah kesimpulan analisis **hasil diabetes**.

---
Catatan:
- Jika terdapat perubahan jawaban pada poin 1—5 (contoh: perbedaan penanganan _outlier_), jelaskan pada laporan mengenai jawaban sebelum, jawaban sesudah, dan alasan pengubahan jawaban.
- Eksperimen dapat berupa penggantian model klasifikasi, pengaturan hyperparameter, model stacking, grid search, oversampling, undersampling, dan lain sebagainya. Semakin variatif eksperimen yang dilakukan, semakin baik.

## Dataset
`diabetes.csv` merupakan dataset yang telah dimodifikasi dari [Diabetes Health Indicators Dataset](https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset/) sebagai kumpulan indikator individu yang diperoleh dari survei untuk kasus diabetes. Dataset ini berguna untuk melakukan prediksi diabetes sehingga suatu individu dapat diketahui memiliki risiko tinggi diabetes atau tidak. Hal ini diperoleh dari fitur-fitur yang dapat dianalisis lebih lanjut sebelum mencapai kesimpulan.

Berikut adalah deskripsi singkat setiap kolom:

1. **HighBP**: Memiliki tekanan darah tinggi (BP: Blood Pressure) atau tidak
2. **HighChol**: Kolesterol tinggi atau tidak
3. **BMI**: Besaran Body Mass Index
4. **Smoker**: Perokok atau bukan perokok
5. **Stroke**: Pernah mengalami struk atau tidak
6. **HeartDiseaseorAttack**: Memiliki riwayat penyakit antara jantung koroner dan serangan jantung atau tidak sama sekali
7. **PhysActivity**: Aktif secara fisik dalam 30 hari terakhir atau tidak
8. **Fruits**: Mengonsumsi buah setiap hari atau tidak
9. **Veggies**: Mengonsumsi sayur setiap hari atau tidak
10. **HvyAlcoholConsump**: Peminum berat alkohol atau bukan
11. **AnyHealthcare**: Memiliki perlindungan kesehatan atau tidak, contohnya memiliki asuransi kesehatan
12. **GenHtlth**: Evaluasi mandiri terhadap kesehatan, skala 1-5 (1: Sangat baik, 2: Cukup Baik, 3: Baik, 4: Biasa saja, 5: Buruk)
13. **MentHlth**: Jumlah hari keadaan mental buruk dalam 30 hari terakhir (skala 0-30 hari)  
14. **PhysHlth**: Jumlah hari keadaan fisik buruk dalam 30 hari terakhir (skala 0-30 hari)
15. **DiffWalk**: Memiliki kesulitan berjalan atau menaiki tangga
16. **Sex**: (M) Male atau (F) Female
17. **Age**: 13 kategori umur (1: 18-24 tahun, 9: 60-64 tahun, 13: 80 tahun ke atas)
18. **Education**: Level edukasi skala 1-6 (1: Tidak pernah sekolah atau hanya TK, 2: SD, dst)
19. **Income**: Skala pendapatan 1-8
20. **Diabetes**: Apakah mengalami diabetes atau tidak (Kolom target)

In [5]:
# Import library di sini
import numpy as np
import pandas as pd
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils._testing import ignore_warnings

In [6]:
data = pd.read_csv("diabetes.csv")
target_col = "Diabetes"

### Encoding and Cleaning for Data

In [7]:
data

,HighBP,HighChol,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes
0,0.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,F,9.0,6.0,8.0,False
1,0.0,0.0,24.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,M,13.0,6.0,8.0,False
2,1.0,0.0,29.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,20.0,5.0,0.0,F,12.0,6.0,8.0,False
3,0.0,0.0,28.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,30.0,0.0,0.0,M,1.0,5.0,7.0,False
4,0.0,0.0,33.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,5.0,30.0,15.0,1.0,F,11.0,4.0,3.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50731,1.0,0.0,23.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,3.0,0.0,0.0,1.0,F,7.0,5.0,8.0,False
50732,1.0,1.0,23.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,1.0,0.0,M,9.0,6.0,8.0,True
50733,1.0,1.0,30.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,2.0,20.0,4.0,1.0,F,7.0,6.0,8.0,True
50734,0.0,0.0,26.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,3.0,3.0,0.0,0.0,F,5.0,6.0,8.0,False


In [8]:
label_encoder = LabelEncoder()
data['Sex'] = label_encoder.fit_transform(data['Sex'])
data


,HighBP,HighChol,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes
0,0.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0,9.0,6.0,8.0,False
1,0.0,0.0,24.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,1,13.0,6.0,8.0,False
2,1.0,0.0,29.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,20.0,5.0,0.0,0,12.0,6.0,8.0,False
3,0.0,0.0,28.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,30.0,0.0,0.0,1,1.0,5.0,7.0,False
4,0.0,0.0,33.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,5.0,30.0,15.0,1.0,0,11.0,4.0,3.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50731,1.0,0.0,23.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,3.0,0.0,0.0,1.0,0,7.0,5.0,8.0,False
50732,1.0,1.0,23.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,1.0,0.0,1,9.0,6.0,8.0,True
50733,1.0,1.0,30.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,2.0,20.0,4.0,1.0,0,7.0,6.0,8.0,True
50734,0.0,0.0,26.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,3.0,3.0,0.0,0.0,0,5.0,6.0,8.0,False


### Train, Test, Val Split

In [9]:
def get_train_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=123)
    return (X_train, X_val, X_test, y_train, y_val, y_test)

In [10]:
X = data.drop(columns="Diabetes")
y = data["Diabetes"].copy()

(X_train, X_val, X_test, y_train, y_val, y_test) = get_train_data(X, y)

# Gunakan data validasi untuk bereksperimen dengan model
# Gunakan data test untuk mengevaluasi model hanya di akhir eksperimen
df_train = pd.concat([X_train, y_train], axis=1)
df_val = pd.concat([X_val, y_val], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

_Deliverable_ yang akan dihasilkan adalah sebagai berikut:
1. berkas _notebook_ dengan format nama `PraktikumIF3270_M1_NIM1_NIM2.ipynb` untuk Bagian 1;
2. berkas _notebook_ dengan format nama `PraktikumIF3270_M2_NIM1_NIM2.ipynb` untuk Bagian 1 + Bagian 2; serta
3. berkas laporan dengan format nama `PraktikumIF3270_NIM1_NIM2.pdf` yang mencakup hal berikut:
    - hasil analisis data,
    - penanganan dari hasil analisis data,
    - justifikasi teknik-teknik yang dipilih,
    - perubahan yang dilakukan pada jawaban poin 1—5 jika ada,
    - desain eksperimen,
    - hasil eksperimen,
    - analisis dari hasil eksperimen,
    - kesimpulan,
    - pembagian tugas/kerja per anggota kelompok

Batas waktu pengumpulan:
- _Deliverable_ poin 1: Senin, 25 April 2023, pukul 12.00 WIB
- _Deliverable_ poin 2: Senin, 25 April 2023, pukul 21.00 WIB
- _Deliverable_ poin 3: Senin, 25 April 2023, pukul 21.00 WIB

### 1. Baseline LogisticRegression Model

In [11]:
@ignore_warnings(category=ConvergenceWarning)
def fit_train_predict_model(X, y, model):
    (X_train, X_val, X_test, y_train, y_val, y_test) = get_train_data(X, y)
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    score = model.score(X_test, y_test)
    confusion_matrix_res = confusion_matrix(y_test, predictions)
    f1_score_res = f1_score(y_test, predictions)

    print("Score:", score)
    print("F1 score:", f1_score_res)
    print("Confusion matrix")
    print(confusion_matrix_res)
    print(cross_val_score(model, X_val, y_val, cv=10))

In [12]:
print("Baseline LogisticRegression Model:")
fit_train_predict_model(X, y, LogisticRegression())

Baseline LogisticRegression Model:
Score: 0.8633228222309814
F1 score: 0.2024151811385854
Confusion matrix
[[8585  147]
 [1240  176]]
[0.87684729 0.87192118 0.86206897 0.86576355 0.8682266  0.86083744
 0.86206897 0.87068966 0.86683107 0.86683107]


### 2. Analisis Data

In [13]:
data

,HighBP,HighChol,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes
0,0.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0,9.0,6.0,8.0,False
1,0.0,0.0,24.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,1,13.0,6.0,8.0,False
2,1.0,0.0,29.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,20.0,5.0,0.0,0,12.0,6.0,8.0,False
3,0.0,0.0,28.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,30.0,0.0,0.0,1,1.0,5.0,7.0,False
4,0.0,0.0,33.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,5.0,30.0,15.0,1.0,0,11.0,4.0,3.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50731,1.0,0.0,23.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,3.0,0.0,0.0,1.0,0,7.0,5.0,8.0,False
50732,1.0,1.0,23.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,1.0,0.0,1,9.0,6.0,8.0,True
50733,1.0,1.0,30.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,2.0,20.0,4.0,1.0,0,7.0,6.0,8.0,True
50734,0.0,0.0,26.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,3.0,3.0,0.0,0.0,0,5.0,6.0,8.0,False


In [14]:
for col in data.columns:
  print(data[col].describe())
  print()

count    50736.000000
mean         0.426916
std          0.494635
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: HighBP, dtype: float64

count    50736.000000
mean         0.420845
std          0.493700
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: HighChol, dtype: float64

count    50736.000000
mean        28.418283
std          6.660023
min         13.000000
25%         24.000000
50%         27.000000
75%         31.000000
max         98.000000
Name: BMI, dtype: float64

count    50736.000000
mean         0.446547
std          0.497139
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: Smoker, dtype: float64

count    50736.000000
mean         0.040267
std          0.196587
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
N

#### Duplicate value

Ditemukan sebanyak 2329 baris duplikat


In [15]:
data.duplicated().sum()

2329

#### Missing value

Tidak ditemukan baris yang memiliki atribut _missing value_

In [16]:
data.isna().sum()

HighBP                  0
HighChol                0
BMI                     0
Smoker                  0
Stroke                  0
HeartDiseaseorAttack    0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
AnyHealthcare           0
GenHlth                 0
MentHlth                0
PhysHlth                0
DiffWalk                0
Sex                     0
Age                     0
Education               0
Income                  0
Diabetes                0
dtype: int64

#### Outliers

Ditemukan outliers pada kolom 'BMI' sebanyak 1979 baris.

Pengecekan outliers hanya dilakukan pada kolom 'BMI'. Kolom lain tidak dilakukan pengecekan outliers karena bertipe _categorical_, _scale_, ataupun _boolean_.

In [17]:
Q1 = data['BMI'].quantile(0.25)
Q3 = data['BMI'].quantile(0.75)
IQR = Q3 - Q1

outlier_threshold = 1.5
low_threshold = Q1 - outlier_threshold * IQR
high_threshold = Q3 + outlier_threshold * IQR

outliers = data[(data['BMI'] < low_threshold) | (data['BMI'] > high_threshold)]

print("BMI outliers count:", len(outliers))


BMI outliers count: 1979


#### Balance of data

Ditemukan bahwa terdapat imbalance data.

In [18]:
data[target_col].value_counts()

Diabetes
False    43790
True      6946
Name: count, dtype: int64

### 3. Dataset Noise Handling Plan

In [19]:
data

,HighBP,HighChol,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes
0,0.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0,9.0,6.0,8.0,False
1,0.0,0.0,24.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,1,13.0,6.0,8.0,False
2,1.0,0.0,29.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,20.0,5.0,0.0,0,12.0,6.0,8.0,False
3,0.0,0.0,28.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,30.0,0.0,0.0,1,1.0,5.0,7.0,False
4,0.0,0.0,33.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,5.0,30.0,15.0,1.0,0,11.0,4.0,3.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50731,1.0,0.0,23.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,3.0,0.0,0.0,1.0,0,7.0,5.0,8.0,False
50732,1.0,1.0,23.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,1.0,0.0,1,9.0,6.0,8.0,True
50733,1.0,1.0,30.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,2.0,20.0,4.0,1.0,0,7.0,6.0,8.0,True
50734,0.0,0.0,26.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,3.0,3.0,0.0,0.0,0,5.0,6.0,8.0,False


Baris data duplikat dapat ditangani dengan menghapus baris-baris tersebut.

In [20]:
data.drop_duplicates(inplace=True)
data

,HighBP,HighChol,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes
0,0.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0,9.0,6.0,8.0,False
1,0.0,0.0,24.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,1,13.0,6.0,8.0,False
2,1.0,0.0,29.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,20.0,5.0,0.0,0,12.0,6.0,8.0,False
3,0.0,0.0,28.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,30.0,0.0,0.0,1,1.0,5.0,7.0,False
4,0.0,0.0,33.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,5.0,30.0,15.0,1.0,0,11.0,4.0,3.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50730,1.0,1.0,28.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,1,10.0,6.0,8.0,False
50731,1.0,0.0,23.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,3.0,0.0,0.0,1.0,0,7.0,5.0,8.0,False
50732,1.0,1.0,23.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,1.0,0.0,1,9.0,6.0,8.0,True
50733,1.0,1.0,30.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,2.0,20.0,4.0,1.0,0,7.0,6.0,8.0,True


Baris yang memuat outlier pada kolom BMI dapat ditangani dengan menghapus baris tersebut.

In [21]:
data.drop(outliers.index, inplace=True)
data

,HighBP,HighChol,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes
0,0.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0,9.0,6.0,8.0,False
1,0.0,0.0,24.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,1,13.0,6.0,8.0,False
2,1.0,0.0,29.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,20.0,5.0,0.0,0,12.0,6.0,8.0,False
3,0.0,0.0,28.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,30.0,0.0,0.0,1,1.0,5.0,7.0,False
4,0.0,0.0,33.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,5.0,30.0,15.0,1.0,0,11.0,4.0,3.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50730,1.0,1.0,28.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,1,10.0,6.0,8.0,False
50731,1.0,0.0,23.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,3.0,0.0,0.0,1.0,0,7.0,5.0,8.0,False
50732,1.0,1.0,23.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,1.0,0.0,1,9.0,6.0,8.0,True
50733,1.0,1.0,30.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,2.0,20.0,4.0,1.0,0,7.0,6.0,8.0,True


Imbalance pada kolom target dapat dilakukan dengan oversampling pada target yang bernilai 'True'. Undersampling sebaiknya tidak dilakukan karena perbandingan jumlah baris data 'False' jauh melebihi baris data 'True'.

In [22]:
data[target_col].value_counts()

Diabetes
False    40157
True      6271
Name: count, dtype: int64

### 4. Encoding

Encoding dilakukan pada kolom Sex dengan cara Label Encoding (telah dilakukan pada proses pembacaan data)
- Nilai 'M' diubah menjadi nilai 1
- Nilai 'F' diubah menjadi nilai 0

Encoding juga dapat dilakukan dengan menormalisasi skala pada kolom-kolom bernilai _scale_.

### 5. Design Experiment

#### Tujuan eksperimen
Membangun model klasifikasi untuk memprediksi apakah seorang individu mengidap penyakit diabetes (ya/tidak) berdasarkan data kesehatan.



#### Variabel dependen dan independen
Variabel dependen:
1. **Diabetes**: apakah seseorang mengalami diabetes atau tidak

Variabel independen:
1. **HighBP**: Memiliki tekanan darah tinggi (BP: Blood Pressure) atau tidak
2. **HighChol**: Kolesterol tinggi atau tidak
3. **BMI**: Besaran Body Mass Index
4. **Smoker**: Perokok atau bukan perokok
5. **Stroke**: Pernah mengalami struk atau tidak
6. **HeartDiseaseorAttack**: Memiliki riwayat penyakit antara jantung koroner dan serangan jantung atau tidak sama sekali
7. **PhysActivity**: Aktif secara fisik dalam 30 hari terakhir atau tidak
8. **Fruits**: Mengonsumsi buah setiap hari atau tidak
9. **Veggies**: Mengonsumsi sayur setiap hari atau tidak
10. **HvyAlcoholConsump**: Peminum berat alkohol atau bukan
11. **AnyHealthcare**: Memiliki perlindungan kesehatan atau tidak, contohnya memiliki asuransi kesehatan
12. **GenHtlth**: Evaluasi mandiri terhadap kesehatan, skala 1-5 (1: Sangat baik, 2: Cukup Baik, 3: Baik, 4: Biasa saja, 5: Buruk)
13. **MentHlth**: Jumlah hari keadaan mental buruk dalam 30 hari terakhir (skala 0-30 hari)  
14. **PhysHlth**: Jumlah hari keadaan fisik buruk dalam 30 hari terakhir (skala 0-30 hari)
15. **DiffWalk**: Memiliki kesulitan berjalan atau menaiki tangga
16. **Sex**: (M) Male atau (F) Female
17. **Age**: 13 kategori umur (1: 18-24 tahun, 9: 60-64 tahun, 13: 80 tahun ke atas)
18. **Education**: Level edukasi skala 1-6 (1: Tidak pernah sekolah atau hanya TK, 2: SD, dst)
19. **Income**: Skala pendapatan 1-8

#### Strategi eksperimen

Strategi eksperimen penyusunan model meliputi pencarian grid search untuk mendapatkan nilai _hyperparameter_ yang optimal.

Strategi eksperimen juga meliputi pengujian model lain, misal RandomForest, dan juga model ensemble.

#### Skema validasi

Validasi model dilakukan dengan menggunakan dataset validation dan dataset test. Kedua dataset ini diperoleh dari proses split data.

Dataset validation digunakan untuk memvalidasi hasil pelatihan model dari dataset training. Validation dilakukan dengan cara _k-fold cross-validation_, dengan jumlah _k=10_.

Dataset test digunakan untuk menguji performa model terhadap data "baru". Dataset test digunakan untuk mengetahui akurasi dan tingkat overfit model terhadap dataset training.

### 6. Implementasi Eksperimen

#### Dataset Normalization

In [23]:
X = data.drop(columns=target_col)
y = data[target_col].copy()
X[X.columns] = MinMaxScaler().fit_transform(X[X.columns])

data = pd.concat([X, y], axis=1)
data

,HighBP,HighChol,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes
0,0.0,1.0,0.370370,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.00,0.000000,0.000000,0.0,0.0,0.666667,1.0,1.000000,False
1,0.0,0.0,0.370370,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.25,0.000000,0.000000,1.0,1.0,1.000000,1.0,1.000000,False
2,1.0,0.0,0.555556,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.25,0.666667,0.166667,0.0,0.0,0.916667,1.0,1.000000,False
3,0.0,0.0,0.518519,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.25,1.000000,0.000000,0.0,1.0,0.000000,0.8,0.857143,False
4,0.0,0.0,0.703704,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.00,1.000000,0.500000,1.0,0.0,0.833333,0.6,0.285714,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50730,1.0,1.0,0.518519,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.50,0.000000,0.000000,0.0,1.0,0.750000,1.0,1.000000,False
50731,1.0,0.0,0.333333,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.50,0.000000,0.000000,1.0,0.0,0.500000,0.8,1.000000,False
50732,1.0,1.0,0.333333,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.25,0.000000,0.033333,0.0,1.0,0.666667,1.0,1.000000,True
50733,1.0,1.0,0.592593,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.25,0.666667,0.133333,1.0,0.0,0.500000,1.0,1.000000,True


#### LogisticRegression Model after Data Cleaning

In [24]:
print("LogisticRegression Model after Data Cleaning:")
fit_train_predict_model(X, y, LogisticRegression())

LogisticRegression Model after Data Cleaning:
Score: 0.867004092181779
F1 score: 0.2158730158730159
Confusion matrix
[[7881  152]
 [1083  170]]
[0.87213997 0.87079408 0.87213997 0.87348587 0.86675639 0.87213997
 0.8654105  0.87483176 0.86137281 0.85983827]



#### Balancing data by oversampling


In [25]:
X_resampled, y_resampled = RandomOverSampler(sampling_strategy="minority").fit_resample(X, y)

print("LogisticRegression Model with Random oversampling:")
fit_train_predict_model(X_resampled, y_resampled, LogisticRegression())

LogisticRegression Model with Random oversampling:
Score: 0.7386540496793874
F1 score: 0.7425803286730438
Confusion matrix
[[5810 2275]
 [1923 6055]]
[0.72395023 0.75252918 0.73618677 0.75019455 0.74785992 0.7229572
 0.73774319 0.73618677 0.73774319 0.72762646]


#### Balancing data by undersampling

In [26]:
X_resampled, y_resampled = RandomUnderSampler().fit_resample(X, y)

print("LogisticRegression Model with Random undersampling:")
fit_train_predict_model(X_resampled, y_resampled, LogisticRegression())

LogisticRegression Model with Random undersampling:
Score: 0.7361498605021921
F1 score: 0.7360446570972887
Confusion matrix
[[924 358]
 [304 923]]
[0.69154229 0.70149254 0.75124378 0.69154229 0.76616915 0.76119403
 0.74626866 0.72       0.735      0.705     ]


#### Balancing data by SMOTE oversampling

In [27]:
X_resampled, y_resampled = SMOTE().fit_resample(X, y)

print("LogisticRegression Model with SMOTE oversampling:")
fit_train_predict_model(X_resampled, y_resampled, LogisticRegression())

LogisticRegression Model with SMOTE oversampling:
Score: 0.747182967067173
F1 score: 0.7515448149281125
Confusion matrix
[[5860 2225]
 [1836 6142]]
[0.74494557 0.74396887 0.75175097 0.75564202 0.7540856  0.73307393
 0.74474708 0.72529183 0.76108949 0.74474708]


#### Balancing data by ADASYN oversampling

In [28]:
X_resampled, y_resampled = ADASYN().fit_resample(X, y)

print("LogisticRegression Model with ADASYN oversampling:")
fit_train_predict_model(X_resampled, y_resampled, LogisticRegression() )

LogisticRegression Model with ADASYN oversampling:
Score: 0.7274451539482835
F1 score: 0.7295313382269903
Confusion matrix
[[5692 2386]
 [1925 5814]]
[0.71958926 0.71879937 0.71090047 0.74624506 0.69802372 0.72964427
 0.72252964 0.71857708 0.72332016 0.74387352]


#### Baseline RandomForestClassifier Model

In [29]:
print("LogisticRegression Model with ADASYN oversampling:")
fit_train_predict_model(X, y, RandomForestClassifier(max_depth=5, random_state=0))

LogisticRegression Model with ADASYN oversampling:
Score: 0.8651733792806375
F1 score: 0.001594896331738437
Confusion matrix
[[8033    0]
 [1252    1]]
[0.86675639 0.86675639 0.86675639 0.8654105  0.8654105  0.8654105
 0.8654105  0.8654105  0.86675639 0.86792453]


#### Baseline DecisionTreeClassifier Model

In [30]:
print("Baseline DecisionTreeClassifier Model:")
fit_train_predict_model(X, y, DecisionTreeClassifier())

Baseline DecisionTreeClassifier Model:
Score: 0.7944217101012276
F1 score: 0.30303030303030304
Confusion matrix
[[6962 1071]
 [ 838  415]]
[0.77927322 0.77927322 0.8115747  0.79273217 0.7846568  0.77792732
 0.8115747  0.79004038 0.81426649 0.79919137]


#### Baseline KNeighborsClassifier Model

In [31]:
print("Baseline KNeighborsClassifier Model:")
fit_train_predict_model(X, y, KNeighborsClassifier(n_neighbors=6))

Baseline KNeighborsClassifier Model:
Score: 0.8587120396295499
F1 score: 0.1402359108781127
Confusion matrix
[[7867  166]
 [1146  107]]
[0.86137281 0.86810229 0.86675639 0.84656797 0.86002692 0.8654105
 0.85868102 0.85733513 0.86137281 0.86522911]


#### Model Ensembling by Voting Classifier

In [32]:
lr = LogisticRegression()
dt = DecisionTreeClassifier()
knn = KNeighborsClassifier(n_neighbors=10)
rf = RandomForestClassifier(max_depth=7)

ensemble = VotingClassifier(estimators=[('lr', lr), ('dt', dt), ('knn', knn), ('rf', rf)], voting='hard')

print("Model Ensembling by Voting Classifier:")
fit_train_predict_model(X, y, ensemble)

Model Ensembling by Voting Classifier:
Score: 0.8661425802283007
F1 score: 0.06471030850263357
Confusion matrix
[[8000   33]
 [1210   43]]
[0.86810229 0.87079408 0.8654105  0.87079408 0.86675639 0.8640646
 0.87079408 0.86944818 0.86675639 0.86927224]


#### Model Ensembling by Stacking Classifier

In [33]:
lr = LogisticRegression()
dt = DecisionTreeClassifier()
knn = KNeighborsClassifier(n_neighbors=10)
rf = RandomForestClassifier(max_depth=7)

ensemble = StackingClassifier(estimators=[('lr', lr), ('dt', dt), ('knn', knn), ('rf', rf)], final_estimator=LogisticRegression())

print("Model Ensembling by Stacking Classifier:")
fit_train_predict_model(X, y, ensemble)

Model Ensembling by Stacking Classifier:
Score: 0.868619427094551
F1 score: 0.26150121065375304
Confusion matrix
[[7850  183]
 [1037  216]]
[0.86944818 0.87213997 0.86675639 0.87213997 0.86810229 0.86810229
 0.86137281 0.87348587 0.86271871 0.85983827]


#### LogisticRegression Hyper Parameter Tuning with GridSearchCV

In [34]:
(X_train, X_val, X_test, y_train, y_val, y_test) = get_train_data(X, y)

params = {
    'C': np.logspace(-3, 3, 20),
    'penalty': ['l1', 'l2']
}

log_reg = LogisticRegression(solver='liblinear', random_state=123)

grid_search = GridSearchCV(log_reg, param_grid=params, cv=10, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("GridSearchCV Result:")
print("Best hyperparameters: ", grid_search.best_params_)
print("Best F1 score: ", grid_search.best_score_)

GridSearchCV Result:
Best hyperparameters:  {'C': 0.1623776739188721, 'penalty': 'l1'}
Best F1 score:  0.8681721309581677


In [35]:
print("LogisticRegression Model after Hyper Parameter:")
best_C = grid_search.best_params_['C']
best_penalty = grid_search.best_params_['penalty']
fit_train_predict_model(X, y, LogisticRegression(C=best_C, penalty=best_penalty, solver='liblinear', random_state=123))

LogisticRegression Model after Hyper Parameter:
Score: 0.8681886711178117
F1 score: 0.2153846153846154
Confusion matrix
[[7894  139]
 [1085  168]]
[0.87079408 0.86810229 0.87213997 0.87617766 0.86675639 0.86944818
 0.86675639 0.87213997 0.86675639 0.86522911]


### 7. Kesimpulan Hasil Eksperimen
Berdasarkan hasil eksperimen menggunakan model LogisticRegression sebagai acuan dasar perbandingan nilai teknik-teknik dan model-model lain kami menemukan bahwa model dengan skor tertinggi adalah model ensembler yang menggabungkan 4 buah model yaitu logistic regression, decision tree, k-nearest neighbors, dan random forest sebagai estimators, dan juga digunakan stacking classifier sebagai teknik untuk menghubungkan keempat estimator yang dipakai. Selain itu percobaan penanganan imbalanced dataset dengan teknik oversampling dan undersampling menghasilkan nilai skor yang lebih rendah namun memiliki F1-score yang lebih besar yang berarti model memiliki nilai precision dan recall yang besar.